### @Author: Amar Pawar ###

### @Date: 2021-09-06 ###

### @Last Modified by: Amar Pawar ###

### @Last Modified time: 2021-09-07 ###

### @Title : Stock price prediction ###

In [1]:
from pyspark.sql import SparkSession
spark= SparkSession.builder.appName('Stock Data').getOrCreate()

2021-09-08 18:10:05,620 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2021-09-08 18:10:11,996 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2021-09-08 18:10:12,001 WARN util.Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
from pyspark.ml.regression import LinearRegression

# Loading file in dataframe

In [3]:
dataset=spark.read.csv("hdfs://localhost:9000/Sample/data.csv",inferSchema=True,header=True)

In [4]:
dataset.columns

['["time"', ' "open"', ' "high"', ' "low"', ' "close"', ' "volume"]']

In [5]:
dataset

DataFrame[["time": string,  "open": string,  "high": string,  "low": string,  "close": string,  "volume"]: string]

# Processing data changing column names

In [4]:
dataset2=dataset.withColumnRenamed('["time"','time')\
    .withColumnRenamed(' "open"','open')\
    .withColumnRenamed(' "high"','high')\
    .withColumnRenamed(' "low"','low')\
    .withColumnRenamed(' "close"','close')\
    .withColumnRenamed(' "volume"]','volume')

In [7]:
dataset2.printSchema()

root
 |-- time: string (nullable = true)
 |-- open: string (nullable = true)
 |-- high: string (nullable = true)
 |-- low: string (nullable = true)
 |-- close: string (nullable = true)
 |-- volume: string (nullable = true)



In [5]:
dataset2.show()

+--------------------+-----------+-----------+-----------+-----------+----------+
|                time|       open|       high|        low|      close|    volume|
+--------------------+-----------+-----------+-----------+-----------+----------+
|["2021-09-03 19:0...|   "139.55"|   "139.55"|   "139.55"|   "139.55"|   "2749"]|
|["2021-09-03 17:4...|   "139.65"|   "139.65"|   "139.65"|   "139.65"|    "150"]|
|["2021-09-03 16:1...|   "139.58"|   "139.61"|   "139.55"|   "139.55"|  "31003"]|
|["2021-09-03 16:0...|  "139.765"|  "139.885"|   "139.54"|   "139.62"| "269779"]|
|["2021-09-03 15:4...|   "139.69"|  "139.769"|  "139.635"|  "139.769"|  "79292"]|
|["2021-09-03 15:3...| "139.8399"|   "139.86"|  "139.655"|   "139.69"|  "49114"]|
|["2021-09-03 15:1...|   "139.76"|   "139.88"|   "139.75"|   "139.83"|  "50153"]|
|["2021-09-03 15:0...|   "139.78"|   "139.84"|   "139.72"|   "139.74"|  "38715"]|
|["2021-09-03 14:4...|   "139.61"|   "139.78"|   "139.59"|   "139.78"|  "31959"]|
|["2021-09-03 14

In [6]:
from pyspark.sql.functions import *

# Processing data removing unneccesary things from data

In [7]:

newDf = dataset2.withColumn('open', regexp_replace('open', '"', ''))\
    .withColumn('time', regexp_replace('time', '\\[', ''))\
    .withColumn('time', regexp_replace('time', '"', ''))\
    .withColumn('high', regexp_replace('high', '"', ''))\
    .withColumn('low', regexp_replace('low', '"', ''))\
    .withColumn('close', regexp_replace('close', '"', ''))\
    .withColumn('volume', regexp_replace('volume', '\\]', ''))\
    .withColumn('volume', regexp_replace('volume', '"', ''))

In [8]:
newDf.show()

+-------------------+---------+---------+---------+---------+-------+
|               time|     open|     high|      low|    close| volume|
+-------------------+---------+---------+---------+---------+-------+
|2021-09-03 19:00:00|   139.55|   139.55|   139.55|   139.55|   2749|
|2021-09-03 17:45:00|   139.65|   139.65|   139.65|   139.65|    150|
|2021-09-03 16:15:00|   139.58|   139.61|   139.55|   139.55|  31003|
|2021-09-03 16:00:00|  139.765|  139.885|   139.54|   139.62| 269779|
|2021-09-03 15:45:00|   139.69|  139.769|  139.635|  139.769|  79292|
|2021-09-03 15:30:00| 139.8399|   139.86|  139.655|   139.69|  49114|
|2021-09-03 15:15:00|   139.76|   139.88|   139.75|   139.83|  50153|
|2021-09-03 15:00:00|   139.78|   139.84|   139.72|   139.74|  38715|
|2021-09-03 14:45:00|   139.61|   139.78|   139.59|   139.78|  31959|
|2021-09-03 14:30:00| 139.6829| 139.6861|   139.57| 139.6311|  31552|
|2021-09-03 14:15:00|   139.63|   139.72|   139.57|   139.69|  34371|
|2021-09-03 14:00:00

In [10]:
import pandas

In [13]:
pandaDf=newDf.toPandas()
pandaDf.isna().sum()

In [12]:
newDf.printSchema()

root
 |-- time: string (nullable = true)
 |-- open: string (nullable = true)
 |-- high: string (nullable = true)
 |-- low: string (nullable = true)
 |-- close: string (nullable = true)
 |-- volume: string (nullable = true)



# Changing datatypes of columns

In [16]:
df2 = newDf.withColumn("open",col("open").cast("double"))\
    .withColumn("high",col("high").cast("double"))\
    .withColumn("low",col("low").cast("double"))\
    .withColumn("close",col("close").cast("double"))\
    .withColumn("volume",col("volume").cast("int"))
    #.withColumn("time1",col("time1").cast("Timestamp"))

In [17]:
df2.show()

+-------------------+--------+--------+--------+--------+------+
|               time|    open|    high|     low|   close|volume|
+-------------------+--------+--------+--------+--------+------+
|2021-09-03 19:00:00|  139.55|  139.55|  139.55|  139.55|  2749|
|2021-09-03 17:45:00|  139.65|  139.65|  139.65|  139.65|   150|
|2021-09-03 16:15:00|  139.58|  139.61|  139.55|  139.55| 31003|
|2021-09-03 16:00:00| 139.765| 139.885|  139.54|  139.62|269779|
|2021-09-03 15:45:00|  139.69| 139.769| 139.635| 139.769| 79292|
|2021-09-03 15:30:00|139.8399|  139.86| 139.655|  139.69| 49114|
|2021-09-03 15:15:00|  139.76|  139.88|  139.75|  139.83| 50153|
|2021-09-03 15:00:00|  139.78|  139.84|  139.72|  139.74| 38715|
|2021-09-03 14:45:00|  139.61|  139.78|  139.59|  139.78| 31959|
|2021-09-03 14:30:00|139.6829|139.6861|  139.57|139.6311| 31552|
|2021-09-03 14:15:00|  139.63|  139.72|  139.57|  139.69| 34371|
|2021-09-03 14:00:00|  139.76|  139.76|  139.63|  139.64| 36656|
|2021-09-03 13:45:00|  13

In [18]:
df2.printSchema()

root
 |-- time: string (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: integer (nullable = true)



In [19]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [49]:
featureassembler=VectorAssembler(inputCols=["open","high","low"],outputCol="Features")

In [50]:
output=featureassembler.transform(df2)

In [51]:
output.show()

+-------------------+--------+--------+--------+--------+------+--------------------+
|               time|    open|    high|     low|   close|volume|            Features|
+-------------------+--------+--------+--------+--------+------+--------------------+
|2021-09-03 19:00:00|  139.55|  139.55|  139.55|  139.55|  2749|[139.55,139.55,13...|
|2021-09-03 17:45:00|  139.65|  139.65|  139.65|  139.65|   150|[139.65,139.65,13...|
|2021-09-03 16:15:00|  139.58|  139.61|  139.55|  139.55| 31003|[139.58,139.61,13...|
|2021-09-03 16:00:00| 139.765| 139.885|  139.54|  139.62|269779|[139.765,139.885,...|
|2021-09-03 15:45:00|  139.69| 139.769| 139.635| 139.769| 79292|[139.69,139.769,1...|
|2021-09-03 15:30:00|139.8399|  139.86| 139.655|  139.69| 49114|[139.8399,139.86,...|
|2021-09-03 15:15:00|  139.76|  139.88|  139.75|  139.83| 50153|[139.76,139.88,13...|
|2021-09-03 15:00:00|  139.78|  139.84|  139.72|  139.74| 38715|[139.78,139.84,13...|
|2021-09-03 14:45:00|  139.61|  139.78|  139.59|  139.

In [23]:
output.select("Features").show()

+--------------------+
|            Features|
+--------------------+
|[139.55,139.55,13...|
|[139.65,139.65,13...|
|[139.58,139.61,13...|
|[139.765,139.885,...|
|[139.69,139.769,1...|
|[139.8399,139.86,...|
|[139.76,139.88,13...|
|[139.78,139.84,13...|
|[139.61,139.78,13...|
|[139.6829,139.686...|
|[139.63,139.72,13...|
|[139.76,139.76,13...|
|[139.69,139.78,13...|
|[139.65,139.73,13...|
|[139.49,139.65,13...|
|[139.46,139.515,1...|
|[139.45,139.47,13...|
|[139.47,139.5,139...|
|[139.4451,139.54,...|
|[139.5201,139.56,...|
+--------------------+
only showing top 20 rows



In [24]:
output.columns

['time', 'open', 'high', 'low', 'close', 'volume', 'Features']

In [25]:
finalized_data=output.select("time","Features","close").sort("time",ascending=True)

In [26]:
finalized_data.show(100)

+-------------------+--------------------+--------+
|               time|            Features|   close|
+-------------------+--------------------+--------+
|2021-08-09 04:15:00|[142.96,143.8,142.5]|   142.5|
|2021-08-09 07:15:00|[142.4,142.63,142.4]|  142.55|
|2021-08-09 07:45:00| [142.7,142.8,142.7]|   142.8|
|2021-08-09 08:00:00| [142.7,142.7,142.7]|   142.7|
|2021-08-09 08:15:00|[142.446,143.2499...|  142.33|
|2021-08-09 08:30:00|[142.27,142.3,142.1]|  142.22|
|2021-08-09 08:45:00| [142.2,142.2,142.2]|   142.2|
|2021-08-09 09:00:00|[142.2,142.35,142...|  142.35|
|2021-08-09 09:15:00|[142.4,142.4,142....|142.3468|
|2021-08-09 09:30:00|[142.34,142.35,14...|  142.05|
|2021-08-09 09:45:00|[142.2,142.495,14...|141.6448|
|2021-08-09 10:00:00|[141.65,141.9998,...| 141.615|
|2021-08-09 10:15:00|[141.62,141.77,14...|  141.55|
|2021-08-09 10:30:00|[141.5594,141.71,...| 141.585|
|2021-08-09 10:45:00|[141.56,141.56,14...|  141.23|
|2021-08-09 11:00:00|[141.25,141.44,14...|  141.09|
|2021-08-09 

In [27]:
train_data,test_data=finalized_data.randomSplit([0.75,0.25])

In [28]:
regressor=LinearRegression(featuresCol='Features', labelCol='close')
regressor=regressor.fit(train_data)

2021-09-07 18:53:19,600 WARN util.Instrumentation: [4a12d183] regParam is zero, which might cause numerical instability and overfitting.
2021-09-07 18:53:21,898 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
2021-09-07 18:53:21,899 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
2021-09-07 18:53:22,049 WARN netlib.LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
2021-09-07 18:53:22,049 WARN netlib.LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [29]:
regressor.coefficients

DenseVector([-0.4976, 0.691, 0.804])

In [30]:
regressor.intercept

0.3710218685987455

In [31]:
pred= regressor.transform(test_data)
pred.select("Features","close","prediction").show()

+--------------------+--------+------------------+
|            Features|   close|        prediction|
+--------------------+--------+------------------+
|[142.446,143.2499...|  142.33|  142.911123256689|
|[141.62,141.77,14...|  141.55|141.49552869329108|
|[141.36,141.46,14...|  141.15|141.16949089208322|
|[141.13,141.2123,...| 141.145|141.14494097528078|
|[141.159,141.29,1...|  141.28|141.22440103386515|
|[141.75,141.81,14...|141.7698|141.69164845643706|
|[141.24,141.37,14...|  141.24|141.28761627202357|
|[141.34,141.34,14...|  141.34|141.34577229823563|
|[141.1001,141.55,...|  141.35|141.41736237226434|
|[141.35,141.35,14...|  141.35| 141.3557464563246|
|[141.35,141.35,14...|  141.35| 141.3557464563246|
|[141.47,141.47,14...|  141.47|141.47543635339235|
|[141.34,141.58,14...|  141.45|141.42718421959358|
|[141.52,141.72,14...|  141.55| 141.6635023545878|
|[141.56,141.643,1...|  141.57|141.46175034762095|
|[141.439,141.44,1...|  141.26|141.24902602130913|
|[141.22,141.48,14...|  141.38|

# Model Evaluating

## RMSC

In [32]:
from pyspark.ml.evaluation import RegressionEvaluator

In [33]:
regression_evaluator= RegressionEvaluator(
predictionCol="prediction",
labelCol="close",
metricName="rmse")
rmse = regression_evaluator.evaluate(pred)
print(f"RMSE is {rmse:.1f}")

RMSE is 0.1


# Final Model

In [36]:
from pyspark.sql import Window

In [37]:
df10=finalized_data.withColumn("rank",percent_rank().over(Window.partitionBy().orderBy("time")))

In [38]:
df10.show()

2021-09-07 18:53:40,609 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------------------+--------------------+--------+--------------------+
|               time|            Features|   close|                rank|
+-------------------+--------------------+--------+--------------------+
|2021-08-09 04:15:00|[142.96,143.8,142.5]|   142.5|                 0.0|
|2021-08-09 07:15:00|[142.4,142.63,142.4]|  142.55|0.001400560224089...|
|2021-08-09 07:45:00| [142.7,142.8,142.7]|   142.8|0.002801120448179...|
|2021-08-09 08:00:00| [142.7,142.7,142.7]|   142.7|0.004201680672268907|
|2021-08-09 08:15:00|[142.446,143.2499...|  142.33|0.005602240896358...|
|2021-08-09 08:30:00|[142.27,142.3,142.1]|  142.22|0.007002801120448...|
|2021-08-09 08:45:00| [142.2,142.2,142.2]|   142.2|0.008403361344537815|
|2021-08-09 09:00:00|[142.2,142.35,142...|  142.35| 0.00980392156862745|
|2021-08-09 09:15:00|[142.4,142.4,142....|142.3468|0.011204481792717087|
|2021-08-09 09:30:00|[142.34,142.35,14...|  142.05|0.012605042016806723|
|2021-08-09 09:45:00|[142.2,142.495,14...|141.6448|

In [39]:
train_df=df10.where("rank<=.8").drop("rank")
test_df=df10.where("rank>.8").drop("rank")

In [40]:
train_df.show()

2021-09-07 18:53:57,155 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------------------+--------------------+--------+
|               time|            Features|   close|
+-------------------+--------------------+--------+
|2021-08-09 04:15:00|[142.96,143.8,142.5]|   142.5|
|2021-08-09 07:15:00|[142.4,142.63,142.4]|  142.55|
|2021-08-09 07:45:00| [142.7,142.8,142.7]|   142.8|
|2021-08-09 08:00:00| [142.7,142.7,142.7]|   142.7|
|2021-08-09 08:15:00|[142.446,143.2499...|  142.33|
|2021-08-09 08:30:00|[142.27,142.3,142.1]|  142.22|
|2021-08-09 08:45:00| [142.2,142.2,142.2]|   142.2|
|2021-08-09 09:00:00|[142.2,142.35,142...|  142.35|
|2021-08-09 09:15:00|[142.4,142.4,142....|142.3468|
|2021-08-09 09:30:00|[142.34,142.35,14...|  142.05|
|2021-08-09 09:45:00|[142.2,142.495,14...|141.6448|
|2021-08-09 10:00:00|[141.65,141.9998,...| 141.615|
|2021-08-09 10:15:00|[141.62,141.77,14...|  141.55|
|2021-08-09 10:30:00|[141.5594,141.71,...| 141.585|
|2021-08-09 10:45:00|[141.56,141.56,14...|  141.23|
|2021-08-09 11:00:00|[141.25,141.44,14...|  141.09|
|2021-08-09 

In [41]:
test_df.show()

2021-09-07 18:53:58,927 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------------------+--------------------+------+
|               time|            Features| close|
+-------------------+--------------------+------+
|2021-08-30 16:00:00|[139.045,139.085,...|138.98|
|2021-08-30 16:15:00|[138.97,138.98,13...|138.97|
|2021-08-30 16:30:00|[139.21,139.21,13...|139.21|
|2021-08-30 16:45:00|[138.81,138.81,13...|138.81|
|2021-08-31 08:15:00|[138.81,139.05,13...|139.05|
|2021-08-31 08:30:00|[139.4355,139.435...| 138.9|
|2021-08-31 09:00:00|[139.25,139.25,13...|139.25|
|2021-08-31 09:15:00|[138.97,138.97,13...|138.97|
|2021-08-31 09:30:00|[139.43,139.59,13...|139.59|
|2021-08-31 09:45:00|[139.54,139.615,1...|139.25|
|2021-08-31 10:00:00|[139.27,139.94,13...|139.66|
|2021-08-31 10:15:00|[139.68,140.5,139...|140.43|
|2021-08-31 10:30:00|[140.445,140.46,1...|140.04|
|2021-08-31 10:45:00|[140.06,140.2404,...|140.06|
|2021-08-31 11:00:00|[140.06,140.33,14...|140.24|
|2021-08-31 11:15:00|[140.27,140.52,14...| 140.3|
|2021-08-31 11:30:00|[140.31,140.505,1...|140.26|


In [ ]:
test_df.write.parquet('testdata')

In [42]:
regressor=LinearRegression(featuresCol='Features', labelCol='close')
lr_model=regressor.fit(train_df)

2021-09-07 18:54:01,089 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2021-09-07 18:54:01,616 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2021-09-07 18:54:02,178 WARN util.Instrumentation: [ad600ae0] regParam is zero, which might cause numerical instability and overfitting.
2021-09-07 18:54:03,252 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [43]:
lr_model.coefficients

DenseVector([-0.4479, 0.6823, 0.7636])

In [44]:
lr_model.intercept

0.28480959021561214

In [45]:
pred= lr_model.transform(test_df)
pred.select("Features","close","prediction").show()

2021-09-07 18:54:07,480 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------+------+------------------+
|            Features| close|        prediction|
+--------------------+------+------------------+
|[139.045,139.085,...|138.98| 139.0088189816052|
|[138.97,138.98,13...|138.97|138.98604284027746|
|[139.21,139.21,13...|139.21|  139.218743620076|
|[138.81,138.81,13...|138.81|138.81953685610827|
|[138.81,139.05,13...|139.05|138.98329554207902|
|[139.4355,139.435...| 138.9|139.03487193320166|
|[139.25,139.25,13...|139.25|139.25866429647274|
|[138.97,138.97,13...|138.97|138.97921956169534|
|[139.43,139.59,13...|139.59|139.53984348571282|
|[139.54,139.615,1...|139.25|139.14872139002136|
|[139.27,139.94,13...|139.66|139.62123962178453|
|[139.68,140.5,139...|140.43|140.24732041147143|
|[140.445,140.46,1...|140.04| 140.0606230490905|
|[140.06,140.2404,...|140.06|140.03742370194436|
|[140.06,140.33,14...|140.24| 140.2207412677876|
|[140.27,140.52,14...| 140.3|140.43195088542484|
|[140.31,140.505,1...|140.26|140.39616197032387|
|[140.24,140.41,14..

In [46]:
regression_evaluator= RegressionEvaluator(
predictionCol="prediction",
labelCol="close",
metricName="rmse")
rmse = regression_evaluator.evaluate(pred)
print(f"RMSE is {rmse:.1f}")

2021-09-07 18:54:09,064 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


RMSE is 0.1


In [ ]:
lr_model.save("stock_model1")

In [48]:
from pyspark.ml.regression import LinearRegressionModel
lrModel=LinearRegressionModel.load("/user/ubuntu/stock_model1")
final_output=lrModel.transform(test_df)
final_output.show()

2021-09-07 18:57:06,631 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------------------+--------------------+------+------------------+
|               time|            Features| close|        prediction|
+-------------------+--------------------+------+------------------+
|2021-08-30 16:00:00|[139.045,139.085,...|138.98| 139.0088189816052|
|2021-08-30 16:15:00|[138.97,138.98,13...|138.97|138.98604284027746|
|2021-08-30 16:30:00|[139.21,139.21,13...|139.21|  139.218743620076|
|2021-08-30 16:45:00|[138.81,138.81,13...|138.81|138.81953685610827|
|2021-08-31 08:15:00|[138.81,139.05,13...|139.05|138.98329554207902|
|2021-08-31 08:30:00|[139.4355,139.435...| 138.9|139.03487193320166|
|2021-08-31 09:00:00|[139.25,139.25,13...|139.25|139.25866429647274|
|2021-08-31 09:15:00|[138.97,138.97,13...|138.97|138.97921956169534|
|2021-08-31 09:30:00|[139.43,139.59,13...|139.59|139.53984348571282|
|2021-08-31 09:45:00|[139.54,139.615,1...|139.25|139.14872139002136|
|2021-08-31 10:00:00|[139.27,139.94,13...|139.66|139.62123962178453|
|2021-08-31 10:15:00|[139.68,140.5